In [2]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lstm
import psutil
import main_lstm
import main_rf
psutil.virtual_memory()

def create_spreads(data):
    #Sort Values and shift to set up collateral requirements
    data['DataDate'] = data.index
    data['DateKey']=data['DataDate'].astype('str')+data['ExpirationDate'].astype('str') 
    
    data['DataDate'] = data['DataDate'].apply(pd.Timestamp)
    data.groupby('DateKey') 
    #data.sort_values('DataDate') 
    data['Prev_Strike']=data['StrikePrice'].shift(1)
    data['Prev_AskPrice']=data['AskPrice'].shift(1)
    data['Prev_DateKey']=data['DateKey'].shift(1)
    data['Prev_xp']=data['xp'].shift(1)
      #Create nan values for the crossover days
    for idx, row in data.iterrows():
        if row['Prev_DateKey']!=row['DateKey']:
            row['Prev_Strike']=np.nan
            row['Prev_AskPrice']=np.nan
            row['Prev_xp']=np.nan
    data['spread_premium'] = data['AskPrice'].astype(float) - data['Prev_AskPrice'].astype(float)
    data['spread_xp'] = data['xp'].astype(float) - data['Prev_xp'].astype(float)
    data['spread_profit'] = data['spread_premium'].astype(float) - data['spread_xp'].astype(float)
    return data     
        # for i in range(len(data)):    
    
#     # Create nan values where the shifted value uses a different day
#     for i in data.index:
#         if data.loc[i, 'Prev_DateKey']!=data.loc[i, 'DateKey']:
#             data.loc[i, 'Prev_Strike']=np.nan
#             data.loc[i, 'Prev_AskPrice']=np.nan
            
#     return data

def buying_puts_blp(df):
    blc = df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]
    #blc['sclog'] = blc['log']
    #blc_total = round(blc['sclog'].sum() * -1)
    blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)

    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs = blc_otm_xs[['log']]
    blc_otm_xs['log'] = (blc_otm_xs['log'] * -1)
    blc_otm_xs.to_csv(f'a{symbol}buying_puts_blp_otm_xs.csv')
    return blc_otm_xs

    blc_otm_xm = blc.loc[blc['otm_xm'] == True]
    blc_otm_xm['log'] = round(blc_otm_xm['log'] * -1)
    blc_otm_xm = blc_otm_xm['log']
#    blc_otm_xm.to_csv(f'bearish_{symbol}buying_puts_blp_otm_xm.csv')

    blc_otm_xf = blc.loc[blc['otm_xf'] == True]
    blc_otm_xf['log'] = round(blc_otm_xf['log'] * -1)
    blc_otm_xf = blc_otm_xf['log']
    blc_otm_xf.to_csv(f'bearish_{symbol}buying_puts_blp_otm_xf.csv')
    
    blc_ctm_xs = blc.loc[blc['ctm_xs'] == True]
    blc_ctm_xs['log'] = round(blc_ctm_xs['log'] * -1)
    blc_ctm_xs = blc_ctm_xs['log']
    blc_ctm_xs.to_csv(f'bearish_{symbol}buying_puts_blp_ctm_xs.csv')
    
    blc_ctm_xf = blc.loc[blc['ctm_xf'] == True]
    blc_ctm_xf['log'] = round(blc_ctm_xf['log'] * -1)
    blc_ctm_xf = blc_ctm_xf['log']
    blc_ctm_xf.to_csv(f'bearish_{symbol}buying_puts_blp_ctm_xf.csv')
    
    blc_ctm_xm = blc.loc[blc['ctm_xm'] == True]
    blc_ctm_xm['log'] = round(blc_ctm_xm['log'] * -1)
    blc_ctm_xm = blc_ctm_xm['log']
    blc_ctm_xm.to_csv(f'bearish_{symbol}buying_puts_blp_ctm_xm.csv')
    #blc_ctm_xf['log'] = blc_ctm_xf['log']
    
    #blc_ctm_xf['log'] = round(blc_ctm_xf['log'])
    #blc_ctm_xf_total = blc_ctm_xf[:, 0].sum()

    print(f'blindly buying puts on bollinger and call signals would net:{blc_total}')
    #print(f'buying close to the money, expiring far calls on bollinger and call signals would net: {blc_ctm_xf_total}' )
    

def selling_put_spreads_blp(df):
    blc = df.loc[(df['bollinger_signal']== 1) & (df['Call signals'] == 1)]
    blc['log'] = blc['spread_profit']
    blc_total = round(blc['log'].sum())
    
    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs['spread_profit'] = round(blc_otm_xs['spread_profit'])
    blc_otm_xs = blc_otm_xs['spread_profit']
    blc_otm_xs.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_otm_xs.csv')
    
    blc_otm_xm = blc.loc[blc['otm_xm'] == True]
    blc_otm_xm['spread_profit'] = round(blc_otm_xm['spread_profit'])
    blc_otm_xm = blc_otm_xm['spread_profit']
    blc_otm_xm.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_otm_xm.csv')

    blc_otm_xf = blc.loc[blc['otm_xf'] == True]
    blc_otm_xf['spread_profit'] = round(blc_otm_xf['spread_profit'])
    blc_otm_xf = blc_otm_xf['spread_profit']
    blc_otm_xf.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_otm_xf.csv')
    
    blc_ctm_xs = blc.loc[blc['ctm_xs'] == True]
    blc_ctm_xs['spread_profit'] = round(blc_ctm_xs['spread_profit'])
    blc_ctm_xs = blc_ctm_xs['spread_profit']
    blc_ctm_xs.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_ctm_xs.csv')
    
    blc_ctm_xf = blc.loc[blc['ctm_xf'] == True]
    blc_ctm_xf['spread_profit'] = round(blc_ctm_xf['spread_profit'])
    blc_ctm_xf = blc_ctm_xf['spread_profit']
    blc_ctm_xf.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_ctm_xf.csv')
    
    blc_ctm_xm = blc.loc[blc['ctm_xm'] == True]
    blc_ctm_xm['spread_profit'] = round(blc_ctm_xm['spread_profit'])
    blc_ctm_xm = blc_ctm_xm['spread_profit']
    blc_ctm_xm.to_csv(f'bullish_{symbol}selling_puts_spreads_blc_ctm_xm.csv')
    #blc_ctm_xf['spread_profit'] = blc_ctm_xf['spread_profit']
    
    #blc_ctm_xf['spread_profit'] = round(blc_ctm_xf['spread_profit'])
    #blc_ctm_xf_total = blc_ctm_xf[:, 0].sum()

    print(f'blindly selling put spreads on bollinger and call signals would net:{blc_total}')
    #print(f'buying close to the money, expiring far calls on bollinger and call signals would net: {blc_ctm_xf_total}' )
    

def selling_puts_blp(df):
    blc = df.loc[(df['bollinger_signal']== 1) & (df['Call signals'] == 1)]
    blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)
    
    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs['log'] = round(blc_otm_xs['log'])
    blc_otm_xs = blc_otm_xs['log']
    blc_otm_xs.to_csv(f'bullish_{symbol}selling_puts_blc_otm_xs.csv')
    
    blc_otm_xm = blc.loc[blc['otm_xm'] == True]
    blc_otm_xm['log'] = round(blc_otm_xm['log'])
    blc_otm_xm = blc_otm_xm['log']
    blc_otm_xm.to_csv(f'bullish_{symbol}selling_puts_blc_otm_xm.csv')

    blc_otm_xf = blc.loc[blc['otm_xf'] == True]
    blc_otm_xf['log'] = round(blc_otm_xf['log'])
    blc_otm_xf = blc_otm_xf['log']
    blc_otm_xf.to_csv(f'bullish_{symbol}selling_puts_blc_otm_xf.csv')
    
    blc_ctm_xs = blc.loc[blc['ctm_xs'] == True]
    blc_ctm_xs['log'] = round(blc_ctm_xs['log'])
    blc_ctm_xs = blc_ctm_xs['log']
    blc_ctm_xs.to_csv(f'bullish_{symbol}selling_puts_blc_ctm_xs.csv')
    
    blc_ctm_xf = blc.loc[blc['ctm_xf'] == True]
    blc_ctm_xf['log'] = round(blc_ctm_xf['log'])
    blc_ctm_xf = blc_ctm_xf['log']
    blc_ctm_xf.to_csv(f'bullish_{symbol}selling_puts_blc_ctm_xf.csv')
    
    blc_ctm_xm = blc.loc[blc['ctm_xm'] == True]
    blc_ctm_xm['log'] = round(blc_ctm_xm['log'])
    blc_ctm_xm = blc_ctm_xm['log']
    blc_ctm_xm.to_csv(f'bullish_{symbol}selling_puts_blc_ctm_xm.csv')
    #blc_ctm_xf['log'] = blc_ctm_xf['log']
    
    #blc_ctm_xf['log'] = round(blc_ctm_xf['log'])
    #blc_ctm_xf_total = blc_ctm_xf[:, 0].sum()

    print(f'blindly selling puts on bollinger and call signals would net:{blc_total}')
    #print(f'buying close to the money, expiring far calls on bollinger and call signals would net: {blc_ctm_xf_total}' )
    

def test_puts(df):
    buying_puts_blp(df)
    selling_put_spreads_blp(df)
    selling_puts_blp(df)

In [ ]:
#tech: FB, GOOG, MSFT,TSLA, AAPL
#etf: MTUM, NDAQ, SPY, VOO
#food: CPG, PG, SBUX
#finance: JPM, GS
#materials: BHP, RIO, VALE
#media: DIS, NFLX, TWTR
#military: BA, LMT,

In [99]:
tf = buying_puts_blp(df)
tf

,log
DataDate,


In [103]:
df

,optionkey,Symbol,ExpirationDate,AskPrice,PutCall,StrikePrice,UnderlyingPrice,time delta,% out,exp?,...,ctm_xf,otm_xf,ftm_xf,atm_xm,ctm_xm,otm_xm,ftm_xm,atm_xs,ctm_xs,otm_xs
DataDate,,,,,,,,,,,,,,,,,,,,,
2016-02-26,TSLA2016-02-26p5.00,TSLA,2016-02-26,0.13,put,5.0,190.4,0 days,-0.973739,True,...,False,False,False,False,False,False,False,False,False,False
2016-02-26,TSLA2016-02-26p10.00,TSLA,2016-02-26,0.13,put,10.0,190.4,0 days,-0.947479,True,...,False,False,False,False,False,False,False,False,False,False
2016-02-26,TSLA2016-02-26p15.00,TSLA,2016-02-26,0.13,put,15.0,190.4,0 days,-0.921218,True,...,False,False,False,False,False,False,False,False,False,False
2016-02-26,TSLA2016-02-26p20.00,TSLA,2016-02-26,0.13,put,20.0,190.4,0 days,-0.894958,True,...,False,False,False,False,False,False,False,False,False,False
2016-02-26,TSLA2016-02-26p25.00,TSLA,2016-02-26,0.05,put,25.0,190.4,0 days,-0.868697,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30,TSLA2022-01-21P840.00,TSLA,2022-01-21,438.00,put,840.0,414.7,753 days,1.025561,False,...,False,False,False,False,False,False,False,False,False,False
2019-12-30,TSLA2022-01-21P85.00,TSLA,2022-01-21,5.65,put,85.0,414.7,753 days,-0.795033,False,...,False,False,False,False,False,False,False,False,False,False
2019-12-30,TSLA2022-01-21P850.00,TSLA,2022-01-21,447.50,put,850.0,414.7,753 days,1.049674,False,...,False,False,False,False,False,False,False,False,False,False


In [110]:
df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]

,optionkey,Symbol,ExpirationDate,AskPrice,PutCall,StrikePrice,UnderlyingPrice,time delta,% out,exp?,...,ctm_xf,otm_xf,ftm_xf,atm_xm,ctm_xm,otm_xm,ftm_xm,atm_xs,ctm_xs,otm_xs
DataDate,,,,,,,,,,,,,,,,,,,,,
2016-03-08,TSLA2016-03-11p65.00,TSLA,2016-03-11,0.13,put,65.0,202.43,3 days,-0.678901,False,...,False,False,False,False,False,False,False,False,False,False
2016-03-08,TSLA2016-03-11p70.00,TSLA,2016-03-11,0.13,put,70.0,202.43,3 days,-0.654201,False,...,False,False,False,False,False,False,False,False,False,False
2016-03-08,TSLA2016-03-11p75.00,TSLA,2016-03-11,0.13,put,75.0,202.43,3 days,-0.629502,False,...,False,False,False,False,False,False,False,False,False,False
2016-03-08,TSLA2016-03-11p80.00,TSLA,2016-03-11,0.05,put,80.0,202.43,3 days,-0.604802,False,...,False,False,False,False,False,False,False,False,False,False
2016-03-08,TSLA2016-03-11p85.00,TSLA,2016-03-11,0.13,put,85.0,202.43,3 days,-0.580102,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-17,TSLA2022-01-21P75.00,TSLA,2022-01-21,10.40,put,75.0,261.97,827 days,-0.713708,False,...,False,False,False,False,False,False,False,False,False,False
2019-10-17,TSLA2022-01-21P80.00,TSLA,2022-01-21,11.35,put,80.0,261.97,827 days,-0.694622,False,...,False,False,False,False,False,False,False,False,False,False
2019-10-17,TSLA2022-01-21P85.00,TSLA,2022-01-21,12.35,put,85.0,261.97,827 days,-0.675535,False,...,False,False,False,False,False,False,False,False,False,False


In [109]:
blc = df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]
blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)
blc_otm_xs = blc.loc[blc['otm_xs'] == True]
blc_otm_xs

,optionkey,Symbol,ExpirationDate,AskPrice,PutCall,StrikePrice,UnderlyingPrice,time delta,% out,exp?,...,otm_xf,ftm_xf,atm_xm,ctm_xm,otm_xm,ftm_xm,atm_xs,ctm_xs,otm_xs,log
DataDate,,,,,,,,,,,,,,,,,,,,,


In [98]:
def buying_puts_blp(df):
    blc = df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]
    #blc['sclog'] = blc['log']
    #blc_total = round(blc['sclog'].sum() * -1)
    blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)

    blc_otm_xs = blc.loc[blc['otm_xs'] == True]
    blc_otm_xs = blc_otm_xs[['log']]
   # blc_otm_xs['log'] = (blc_otm_xs['log'] * -1)
    #blc_otm_xs.to_csv(f'a{symbol}buying_puts_blp_otm_xs.csv')
    return blc_otm_xs

In [111]:
sector = 'tech'
symbol = 'TSLA'
df = pd.read_pickle(f'aaafinally_{sector}_{symbol}_puts.pickle')
                  

In [119]:
def puts_blp(df):
    blc = df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]
    #blc['sclog'] = blc['log']
    #blc_total = round(blc['sclog'].sum() * -1)
    blc['log'] = blc['AskPrice'].astype(float) - blc['xp'].astype(float)

    blc_otm_xs = blc.loc[(blc['time delta'] < np.timedelta64(8, 'D')) & (blc['% out'] > -5) & (blc['% out'] < -.2)]
    blc_otm_xs = blc_otm_xs[['log']]
    blc_otm_xs['log'] = (blc_otm_xs['log'] * -1)
    blc_otm_xs.to_csv(f'a{symbol}buying_puts_blp_otm_xs.csv')
    return blc_otm_xs

In [115]:
df.loc[df['otm_xf'] == True]

,optionkey,Symbol,ExpirationDate,AskPrice,PutCall,StrikePrice,UnderlyingPrice,time delta,% out,exp?,...,ctm_xf,otm_xf,ftm_xf,atm_xm,ctm_xm,otm_xm,ftm_xm,atm_xs,ctm_xs,otm_xs
DataDate,,,,,,,,,,,,,,,,,,,,,


In [120]:
ty = puts_blp(df)
ty

,log
DataDate,
2016-03-08,-0.06
2016-03-08,1.48
2016-03-08,0.01
2016-03-08,-0.00
2016-03-08,-0.00
...,...
2019-10-17,-0.00
2019-10-17,-0.00
2019-10-17,-0.00


In [94]:
df['otm_xf'].value_counts()

False    1025915
Name: otm_xf, dtype: int64

In [80]:
df = df.loc[(df['bollinger_signal']== -1) & (df['Put signals'] == 1)]
df['log'] = df['AskPrice'].astype(float) - df['xp'].astype(float)
df

,optionkey,Symbol,ExpirationDate,AskPrice,PutCall,StrikePrice,UnderlyingPrice,time delta,% out,exp?,...,otm_xf,ftm_xf,atm_xm,ctm_xm,otm_xm,ftm_xm,atm_xs,ctm_xs,otm_xs,log
DataDate,,,,,,,,,,,,,,,,,,,,,
2016-03-07,BA2016-03-11p85.00,BA,2016-03-11,0.07,put,85.0,122.91,4 days,-0.308437,False,...,False,False,False,False,False,False,False,False,False,0.06
2016-03-07,BA2016-03-11p90.00,BA,2016-03-11,0.01,put,90.0,122.91,4 days,-0.267757,False,...,False,False,False,False,False,False,False,False,False,0.00
2016-03-07,BA2016-03-11p92.50,BA,2016-03-11,0.01,put,92.5,122.91,4 days,-0.247417,False,...,False,False,False,False,False,False,False,False,False,-0.05
2016-03-07,BA2016-03-11p95.00,BA,2016-03-11,0.01,put,95.0,122.91,4 days,-0.227077,False,...,False,False,False,False,False,False,False,False,False,0.00
2016-03-07,BA2016-03-11p96.00,BA,2016-03-11,0.06,put,96.0,122.91,4 days,-0.218941,False,...,False,False,False,False,False,False,False,False,False,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-25,BA2019-01-18p345.00,BA,2019-01-18,96.10,put,345.0,254.30,480 days,0.356665,False,...,False,False,False,False,False,False,False,False,False,96.07
2017-09-25,BA2019-01-18p350.00,BA,2019-01-18,101.10,put,350.0,254.30,480 days,0.376327,False,...,False,False,False,False,False,False,False,False,False,101.08
2017-09-25,BA2019-01-18p355.00,BA,2019-01-18,106.00,put,355.0,254.30,480 days,0.395989,False,...,False,False,False,False,False,False,False,False,False,105.99


In [81]:
df['log'].nunique()

3392

In [82]:
df.loc[df['otm_xs'] == True]

,optionkey,Symbol,ExpirationDate,AskPrice,PutCall,StrikePrice,UnderlyingPrice,time delta,% out,exp?,...,otm_xf,ftm_xf,atm_xm,ctm_xm,otm_xm,ftm_xm,atm_xs,ctm_xs,otm_xs,log
DataDate,,,,,,,,,,,,,,,,,,,,,
